# Loop over fields and make html overview pages

In [1]:
import yaml
import glob
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from pymoc import MOC
import healpy as hp

from herschelhelp_internal.utils import flux_to_mag

In [2]:
def field_to_sex(moc_file):
    "Take a moc and return the median ra dec"
    m = MOC(filename=moc_file)
    ra, dec = [],[]
    
    for cell in m.flattened(7):
        
        theta,phi = hp.pix2ang(2**7, cell, nest=True)

        dec.append((0.5*np.pi - theta) * 180/np.pi)
        ra.append(phi* 180/np.pi)
        
        

    c = SkyCoord(np.median(ra), np.median(dec),frame='icrs', unit='deg')
    
    return c.to_string('hmsdms').replace('h',' ').replace('m',' ').replace('d',' ').replace('s',' ')
    
#field_to_sex(yml['region'].replace('dmu_products','../../'))

In [3]:
meta_ymls = glob.glob('../../dmu32/*/meta_main.yml')

In [5]:
hedam='http://hedam.lam.fr/HELP/dataproducts'

In [6]:
overview=Table.read('../../dmu32/dr1_data_products_overview.csv')
depth_overviews=Table.read('../../dmu32/dmu32_AllSky/dr1_depth_overviews.csv')

In [7]:
depth_overviews

field,ferr_u_mean_median,ferr_g_mean_median,ferr_r_mean_median,ferr_i_mean_median,ferr_z_mean_median,ferr_y_mean_median,ferr_j_mean_median,ferr_h_mean_median,ferr_k_mean_median,ferr_ks_mean_median,ferr_irac_i1_mean_median,ferr_irac_i2_mean_median,ferr_irac_i3_mean_median,ferr_irac_i4_mean_median,ferr_mips_24_mean_median,ferr_pacs_green_mean_median,ferr_pacs_red_mean_median,ferr_spire_250_mean_median,ferr_spire_350_mean_median,ferr_spire_500_mean_median
str18,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
SA13,nan,0.499644935131073,0.7986563444137573,nan,0.9065105319023132,nan,14.501233100891113,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3705.364013671875,4037.108642578125,4141.72412109375
Bootes,0.10898130033345059,0.3188217580318451,0.19392904215934278,0.40504239216048094,0.3396558149016761,1.15686392984606,1.8290149834742848,2.433102216580356,4.946307573339122,23.00567387582105,1.3749062614269807,1.8823400908678791,10.836242823983468,15.135585643717517,16.38863754272461,9715.158203125,13206.236328125,3246.1162109375,3740.114990234375,5223.958984375
xFLS,2.5981874465942383,0.577968418598175,0.16221551597118378,1.590234294227871,2.751865564430781,6.875311435130419,11.35222053527832,nan,nan,nan,7.403669558268787,8.833639499169525,29.367724531714284,29.851775776160004,9.724309921264648,nan,nan,3286.6875,3642.482666015625,4220.560546875
GAMA-15,0.4427473247051239,0.12072403728961945,0.12575195729732513,0.48367467522621155,0.452085018157959,3.2316691875457764,3.6990435123443604,6.410440921783447,12.525550842285156,7.522132396697998,nan,nan,nan,nan,nan,19691.93359375,18558.060546875,5106.9755859375,5166.611328125,6490.95751953125
GAMA-12,0.41562238335609436,0.13120463490486145,0.13316823542118073,0.5314038991928101,0.39746949076652527,3.540733575820923,4.398492336273193,7.363485813140869,12.678882598876953,8.253340721130371,nan,nan,nan,nan,nan,21138.68359375,19303.3671875,5076.7255859375,5106.244140625,6284.501953125
HDF-N,0.06014675394855649,1.3818777735856416,1.1716443860025318,1.6228076050734046,2.9377319800322845,5.8660030659659,nan,nan,0.05740212276214833,0.4566996904024768,0.10814776693584569,0.10191452447040789,0.692944523239918,0.675048084995405,nan,nan,nan,2362.53662109375,2806.37451171875,2727.918212890625
HATLAS-NGP,nan,1.1068435373359502,1.1280449351446002,1.346519620469588,0.8371601700782776,5.200235573175303,6.290984153747559,10.445904731750488,11.602095603942871,nan,nan,nan,nan,nan,nan,19003.72265625,18367.27734375,5174.78125,5124.71240234375,5790.87890625
ELAIS-N2,0.021648168563842773,0.021362041543176936,0.029633822046084647,0.07714027911424637,0.1036979019628619,5.692292634532179,nan,nan,nan,nan,0.8554748603351955,1.1536211920529802,4.81047619047619,5.009145768025078,14.556729316711426,23417.884765625,14200.380859375,3349.41748046875,3733.767578125,4536.9248046875
EGS,0.08380091767084885,0.04328254815720886,0.0690450668334961,0.12278351160752989,0.24312596023082733,5.537406787291107,12.964601516723633,0.17524227621391958,2.621839515379559,0.2610806312441928,0.47927530201342283,0.44784326769525834,1.3045725,1.5293367837692826,nan,5127.20068359375,6389.46826171875,nan,8507.791015625,13703.501953125


In [8]:
overview

field,objects,dr1_file,dr1_file_hash,area_sq_degrees,file_size_bytes,file_size_readable,xid_objects,photoz_objects,cigale_objects
str18,int64,str81,str32,float64,int64,str9,int64,int64,int64
AKARI-NEP,531746,dmu_products/dmu32/dmu32_AKARI-NEP/data/AKARI-NEP_20180215.fits,5b33074e816785e05407952f03e78354,9.194732358779467,433440000,433.44 MB,31441,107228,1239
AKARI-SEP,844172,dmu_products/dmu32/dmu32_AKARI-SEP/data/AKARI-SEP_20180221.fits,56889960f654742f80329e8ebb0053d5,8.713306475131118,668652480,668.65 MB,108119,139059,566
Bootes,3398098,dmu_products/dmu32/dmu32_Bootes/data/Bootes_20190701.fits,6d2f18ef25a42999d805b258c751fde2,11.42815299095486,5556003840,5.56 GB,495159,1570512,38980
CDFS-SWIRE,2171051,dmu_products/dmu32/dmu32_CDFS-SWIRE/data/CDFS-SWIRE_20180613.fits,7bb01da6145d35ffa157b7c3ee0209ce,12.971246403717068,6059603520,6.06 GB,73511,136944,9308
COSMOS,2599374,dmu_products/dmu32/dmu32_COSMOS/data/COSMOS_20190402.fits,c900763f01cd327b84fafa3defe46151,5.083863478496816,10954031040,10.95 GB,25898,691502,15747
EGS,1412613,dmu_products/dmu32/dmu32_EGS/data/EGS_20180501.fits,74ce7e6bd7a982141d7558fad62b38df,3.566383275122158,4018965120,4.02 GB,223598,1182503,4159
ELAIS-N1,4026292,dmu_products/dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171016.fits,97ccb7d86c92aa9e9ba0657c9e737276,13.507484555454763,6007360320,6.01 GB,269611,2714686,49985
ELAIS-N2,1783240,dmu_products/dmu32/dmu32_ELAIS-N2/data/ELAIS-N2_20180218.fits,09628a8b89fd5eeea27d6059988eb900,9.167479903991113,2321884800,2.32 GB,86591,120723,6798
ELAIS-S1,1655564,dmu_products/dmu32/dmu32_ELAIS-S1/data/ELAIS-S1_20180416.fits,e1e31bd8ae8b7646c47aaec6a40e2cf4,9.002940646885508,2231798400,2.23 GB,194276,1013582,25393


In [9]:
regions = yaml.load(open('../../dmu2/meta_main.yml', 'r'))

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [15]:
template = open('./html.template', 'r').read()
for field_meta in meta_ymls:
    yml = yaml.load(open(field_meta, 'r'))
    print(yml['field'])
    
    description = yml['field']
    for f in regions['fields']:
        if f['name'] == yml['field']:
            description = f['description']
    #make html_table of surveys:
    survey_table = '<ul>' 
    for s in yml['surveys']:
        survey_table+='<li> <a href="https://github.com/H-E-L-P/dmu_products/tree/master/dmu0/dmu0_{}">{}</a></li>'.format(
            s, s
        ) 
    survey_table+= '</ul>'

    median_g_depth=depth_overviews[depth_overviews['field'] ==yml['field']]['ferr_g_mean_median'][0]
    median_k_depth=np.nanmin([
                    depth_overviews[depth_overviews['field'] ==yml['field']]['ferr_ks_mean_median'][0],
                    depth_overviews[depth_overviews['field'] ==yml['field']]['ferr_k_mean_median'][0]
                ])
    median_i1_depth=depth_overviews[depth_overviews['field'] ==yml['field']]['ferr_irac_i1_mean_median'][0]
    median_250_depth=depth_overviews[depth_overviews['field'] ==yml['field']]['ferr_spire_250_mean_median'][0]
    field_html = open('./' + yml['field'] + '.html', "w+")
    field_html.write(template.format(
        field=yml['field'],
        field_description=description,
        target = field_to_sex(yml['region'].replace('dmu_products','../../')), # '10 45 00 +45 00 00',
        fov=3*np.sqrt(overview[overview['field'] == yml['field']]['area_sq_degrees'][0]),
        area=round(overview[overview['field'] == yml['field']]['area_sq_degrees'][0],2),
        masterist_objects=overview[overview['field'] == yml['field']]['objects'][0],
        xid_objects=overview[overview['field'] == yml['field']]['xid_objects'][0],
        photoz_objects=overview[overview['field'] == yml['field']]['photoz_objects'][0],
        cigale_objects=overview[overview['field'] == yml['field']]['cigale_objects'][0],
        median_g_depth="AB {} mag, or {} uJy".format(
            round(flux_to_mag(median_g_depth*1.e-6)[0],2),
            round(median_g_depth,2)),
        median_k_depth="AB {} mag, or {} uJy".format(
            round(flux_to_mag(median_k_depth*1.e-6)[0],2),
            round(median_k_depth,2)),
        median_i1_depth="AB {} mag, or {} uJy".format(
            round(flux_to_mag(median_i1_depth*1.e-6)[0],2),
            round(median_i1_depth,2)),
        median_250_depth="AB {} mag, or {} uJy".format(
            round(flux_to_mag(median_250_depth*1.e-6)[0],2),
            round(median_250_depth,2)),
        best_phot=hedam + '/dmu32/dmu32_AllSky/data/{}_best_phot_20201111.fits'.format(yml['field']),
        master_list=hedam+yml['final'].replace('dmu_products',''),
        blind_list=(
            hedam+
            '/dmu22/dmu22_{}/data/dmu22_XID+SPIRE_{}_BLIND_Matched_MF.fits'.format(
                yml['field'],yml['field']
            )
        ),
        spire_250=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE250_v1.0.fits'.format(yml['field']),
        spire_350=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE350_v1.0.fits'.format(yml['field']), 
        spire_500=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE500_v1.0.fits'.format(yml['field']),
        photoz_file=hedam+yml['photoz'],
        cigale_file=hedam+yml['cigale'],
        seds=hedam+'/dmu28/dmu28_{}/data/zphot/SEDs/'.format(yml['field']),
        moc=hedam+yml['region'],
        survey_table=survey_table
    ))
    field_html.close()

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


COSMOS
ELAIS-S1
GAMA-09
Lockman-SWIRE
ELAIS-N2
Herschel-Stripe-82
SA13
GAMA-12


/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: All-NaN axis encountered


GAMA-15
AKARI-SEP
HATLAS-SGP
xFLS
CDFS-SWIRE
SSDF
XMM-LSS
SPIRE-NEP
AKARI-NEP
ELAIS-N1
HATLAS-NGP
XMM-13hr
Bootes
HDF-N
EGS
